# Coventry

Uses the Rotation pattern to generate the Plain Hunt Minimus change-ringing pattern for 10 bells from the old Cathedral Church of St. Michael in Coventry, England. See: https://www.hibberts.co.uk/coventry-cathedral-old-ten/ for more information.

Running this demo requires a jupyter kernel (runtime environment) that contains the musx package.  See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for directions on how to install musx in your environment.
<hr style="height:1px; color: grey;">

Python imports:

In [ ]:
from musx import version, Note, Rotation, Seq, MidiFile, Score, keynum, setmidiplayer, playfile
from musx.midi.gm import Celesta, Glockenspiel, MusicBox, Vibraphone,\
        Marimba, Xylophone, TubularBells
print(f"musx version: {version}")

Generate midi files and automatically play them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal based midi player to use with musx.  If you dont have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf3")

The old Coventry Cathedral (no longer standing) had 10 bells, represented below as the letters 'a' to 'j' with 'a' being the highest bell.  Rows are bell harmonics, with the 'prime' harmonic being the main tone in each bell. Bell pitches are in Hertz.

In [ ]:
coventry_data = {
#         hum    prime  tierce quint  nominal superq  octnom
    'a': [377,   620.5, 825.5, 1162,  1376,   2032.5, 2753.5],
    'b': [345.5, 577,   750.5, 1064,  1244,   1831,   2483],
    'c': [296,   499,   665,   874,   1114,   1647,   2241],
    'd': [285.5, 483,   626,   855.5, 1044,   1546.5, 2119],
    'e': [261,   432,   564,   760.5, 928,    1366,   1858],
    'f': [234.5, 410,   514,   672,   842,    1239,   1697],
    'g': [201,   360,   444,   598,   740,    1103,   1517],
    'h': [186,   365,   427,   552.5, 695.5,  1025.5, 1404.5],
    'i': [175,   304,   376,   514.5, 616,    908,    1243],
    'j': [159,   283.5, 343,   453.5, 558,    823,    1126]
}
print(f"coventry_data: {coventry_data}")

Convert bell values from Hertz into equivalent floating point key numbers, where 60.5 is 50 cents above middle C:

In [ ]:
coventry_fkeys = {b: [keynum(h, filt=None) for h in l]
                       for b,l in coventry_data.items()}
print(f"coventry_fkeys: {coventry_fkeys}")

Names for the 10 bells, a thru j:

In [ ]:
bells = ['a','b','c','d','e','f','g','h','i','j']
print(f"bells: {bells}")

Two 'swapping rules' (start, stop, width) that produce the pattern:

In [ ]:
rules = [[0, 2, 1], [1, 2, 1]]
print(f"rules: {rules}")

Generate the notes of the complete rotation pattern. The wrapped value causes the end of the pattern to match the start of the pattern. See documentation on Rotation for more information:

In [ ]:
pattern = Rotation(bells, rules).all(wrapped=True)
print(f"length of bell pattern: {len(pattern)}")

The playbells composer:

In [1]:
def playbells(score, peal, belldata, rhy, dur, amp):
    """
    Plays the bell pattern and enmphasizes the first and last bell.

    Parameters
    ----------
    score : Score
        The scheduling queue to run the composer in.
    peal : list
        The bell pattern to play Each bell is represented
        by the letters 'a' to 'j' in the list.
    belldata : dictionary
        Spectral data for each bell.
    rhy : int | float
        The rhythm to use.
    dur : int | float
        The duration for the bells.
    amp : int | float
        The amplitude for the bells.   
    """
    # each bell is represented by its prime bell harmonic.
    primes = {k: belldata[k][1] for k in belldata.keys()}
    # play the peal (the ordered list of bells to play)
    for b in peal:
        # emphasize top and bottom bell by playing all its harmonics.
        if b in ['a','j']: 
            # keynums are quantized to 25 cents
            for k in [x for x in belldata[b]]:
                m = Note(time=score.now, duration=dur*4, pitch=k, amplitude=amp)            
                score.add(m)
        else: # else play single 'prime' note 
            k = primes[b]
            m = Note(time=score.now, duration=dur, pitch=k, amplitude=amp)
            score.add(m)
        yield rhy
print(f"playbells: {playbells}")

playbells: <function playbells at 0x1078d4820>


Define track0 to be a midi meta track that holds tempo, midi instrument assignments, micro tuning, etc.:

In [ ]:
track0 = MidiFile.metatrack(ins={0: TubularBells, 1: TubularBells, 
                               2: TubularBells, 3: TubularBells})
print(f"track0: {track0}")

Track1 will hold the composition:

In [ ]:
track1 = Seq()
print(f"track1: {track1}")

Create a score and pass it track1 to hold the output midi event data:

In [ ]:
score = Score(out=track1)
print(f"score: {score}")

Generate the composition:

In [ ]:
score.compose(playbells(score, pattern, coventry_fkeys, .25, .6, .8))
print("OK!")

Write the tracks to a midi file in the current directory:

In [ ]:
file = MidiFile("coventry.mid", [track0, track1]).write()
print(f"Wrote '{file.pathname}'")

Play the output midi file if a terminal based midi player is installed:

In [ ]:
playfile(file.pathname)